# Korean MongoDB

### Создание базы

In [135]:
import pandas as pd
df = pd.read_csv('ver_b.csv', sep='\t')

In [136]:
df = df.drop(['TRANSLIT', 'AUTO_GLOSS', 'ContKOR', 'RUS', 'COMMENT'], axis=1)[:477]

In [137]:
df

,ID,ADDRESS,KOR,BASE,SUF,GLOSS,TRANS,WClass
0,1,6a 1,夫부와,pwu,wa,COMIT,husband,N
1,2,6a 1,婦부,pwu,non,TOP,wife,N
2,3,6a 1,두,twu,NaN,NaN,two,NUM
3,4,6a 1,姓셩의,syeng,uy,GEN,family,N
4,5,6a 1,合합홈이라,hap-ho,wo=m+i=la,VOL+NMNZ+COP+DECL,unite,N+COP
...,...,...,...,...,...,...,...,...
472,473,13a 7,양짓믈고,yang-cic-mul-ho,kwo,COOR,brush teeth,P
473,474,13a 7,父부母모의,pwu-mwo,uy,GEN,parents,N
474,475,13a 7,고,kwot,oy,LOC,place,BN
475,476,13a 7,가,ka,(a),CV,go,P


In [138]:
df = df.dropna(subset=['ADDRESS', 'KOR'])

In [139]:
import re

page_lines = [re.findall('\d+', pl) for pl in df['ADDRESS'].values]
df['Page'] = [int(a[0]) for a in page_lines]
df['Line'] = [int(a[1]) for a in page_lines]

/home/marynepo/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/marynepo/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [140]:
df['Let_page'] = [re.findall('[ab]', adr)[0] for adr in df['ADDRESS']]

/home/marynepo/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [141]:
pp = 6
pa = 'a'
pl = 1
npl = 1
ls = [1]
for p, a, l in zip(df['Page'][1:], df['Let_page'][1:], df['Line'][1:]):
    if int(p) == pp and int(l) == pl and pa == a:
        ls.append(npl)
    elif int(p) == pp and int(l) != pl and pa == a:
        ls.append(npl + 1)
        npl = npl + 1
    else:
        ls.append(1)
        npl = 1
    pp = p
    pl = l
    pa = a

In [142]:
df['Line'] = ls

/home/marynepo/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [143]:
df[-10:]

,ID,ADDRESS,KOR,BASE,SUF,GLOSS,TRANS,WClass,Page,Line,Let_page
467,468,13a 6,이,tolk,i,NOM,chicken,N,13,6,a
468,469,13a 6,처음,che-um,NaN,NaN,beginning,N,13,6,a
469,470,13a 6,울거든,ul,ke-tun,CONC,cry,P,13,6,a
470,471,13a 6,다,ta,NaN,NaN,all,DET,13,6,a
471,472,13a 6,셰슈며,syey-sywu-ho,mye,COOR,wash one's face,P,13,6,a
472,473,13a 7,양짓믈고,yang-cic-mul-ho,kwo,COOR,brush teeth,P,13,7,a
473,474,13a 7,父부母모의,pwu-mwo,uy,GEN,parents,N,13,7,a
474,475,13a 7,고,kwot,oy,LOC,place,BN,13,7,a
475,476,13a 7,가,ka,(a),CV,go,P,13,7,a
476,477,13a 7,긔운을,kuy-wun,ul,ACC,"strength, energy",N,13,7,a


In [144]:
pp = 6
pl = 1
pa = 'a'
pages_dicts = []
pag_lines = []
line_cur = []
w_pst = []
for w,p,let,l in zip(df['KOR'], df['Page'], df['Let_page'], df['Line']):
    if str(pp) + pa == str(p) + let:
        if pl == int(l):
            line_cur.append(w)
        else:
            pag_lines.append(' '.join(line_cur))
            line_cur = [w]
    else:
        pag_lines.append(' '.join(line_cur))
        pages_dicts.append({'page_num': pp, 'page_let': pa, 'lines': pag_lines})
        pag_lines = []
        line_cur = [w]
    pp = int(p)
    pl = int(l)
    pa = let
pages_dicts.append({'page_num': pp, 'page_let': pa, 'lines': pag_lines})

In [147]:
suf_dicts = []
words_dicts = []
bases_dicts = []
tokens_dicts = []
k = 0
pp = 6
pa = a
pl = 1
for kr, w, tr, s, gl, cl, p, a, l in zip(df['KOR'], df['BASE'], df['TRANS'], df['SUF'], df['GLOSS'], df['WClass'], df['Page'], df['Let_page'], df['Line']):
    if str(p) + a + str(l) != str(pp) + pa + str(pl):
        k = 1
    else:
        k += 1
    if str(s) in ['nan', '']:
        sfs = [('-', '-')]
        gls = ['-']
    else:
        if str(tr) in ['nan', '']:
            tr = '?'
        if str(gl) in ['nan', '']:
            gl = '?'
        if str(cl) in ['nan', '']:
            cl = '?'        
        ss = re.split('\+|=', s)
        gls = gl.split('+')
        try:
            assert len(ss) == len(gls)
        except:
            if gls == ['PROC', 'PARA']:
                ss = ['no', 'ni']
            elif gls == ['SHOULD', 'DECL']:
                ss = ['l-ti-ni', 'la']
            elif gls == ['PROC', 'PIND', 'DRV{N}', 'ACC']:
                ss = ['no', 'n', 'i', 'lul']
            elif gls == ['DECL', 'PIND']:
                ss = ['la', 'n']
            elif gls == ['HONSub', 'QUOT']:
                ss = ['wo-sya', 'toy']
            elif gls == ['SHOULD', 'DECL']:
                ss = ['si-ni', 'la']
            elif gls == ['?']:
                ss = ['le-pse']
            else:
                print(ss, gls)
        sfs = list(zip(ss, gls))
    b_wc = cl
    for g in gls[::-1]:
        if gl == 'NMNZ' or gl == 'DRV{N}':
            b_wc = 'N'
            break
        elif gl == 'ADV':
            b_wc = 'ADV'
            break
        elif gl == 'COP':
            b_wc = 'P'
            break
    bases_dicts.append((w, tr, cl))
    suf_dicts.extend([(suf, g) for suf, g in sfs])
    words_dicts.append((kr, b_wc, bases_dicts[-1], tuple([(suf,g) for suf, g in sfs])))
    tokens_dicts.append({'word': words_dicts[-1], 'page': str(p) + a, 'line': int(l), 'line_pos': k})
    pa = a
    pl = l
    pp = p

In [148]:
bases_dicts = set(bases_dicts)
suf_dicts = set(suf_dicts)
words_dicts = set(words_dicts)
bds = []
sds = []
wds = []
for w, tr, cl in bases_dicts:
    bds.append({'base': w, 'trans': tr, 'pos': cl})
for suf, g in suf_dicts:
    sds.append({'suf': suf, 'gloss': g})
for kr, wc, bd, sg in words_dicts:
    wds.append({'kor': kr, 'pos': wc, 'lemma': bd, 'sufs': sg})

In [149]:
len(bases_dicts)

309

In [150]:
len(suf_dicts)

90

In [151]:
len(words_dicts)

394

In [317]:
import json

with open('suf_dict.json', 'w') as f:
    json.dump(sds, f)

with open('base_dict.json', 'w') as f:
    json.dump(bds, f)

with open('word_dict.json', 'w') as f:
    json.dump(wds, f)

with open('token_dict.json', 'w') as f:
    json.dump(tokens_dicts, f)
    
with open('pages_dict.json', 'w') as f:
    json.dump(pages_dicts, f)

In [381]:
!pip install pymongo==4.4.0

You should consider upgrading via the '/home/marynepo/anaconda3/envs/py37/bin/python -m pip install --upgrade pip' command.


### Подключение к базе

In [445]:
import pymongo
client = pymongo.MongoClient('127.0.0.1', 27017)

db = client['tonmon_sonsup_mongodb']
pages = db['pages']
words = db['words']
bases = db['bases']
tokens = db['tokens']
sufs = db['sufixes']

In [446]:
with open('suf_dict.json', 'r') as f:
    sfs = json.load(f)

with open('base_dict.json', 'r') as f:
    bs = json.load(f)

with open('word_dict.json', 'r') as f:
    ws = json.load(f)

with open('token_dict.json', 'r') as f:
    tks = json.load(f)
    
with open('pages_dict.json', 'r') as f:
    pgs = json.load(f)

### Insert

In [447]:
pages.insert_many(pgs)
bases.insert_many(bs)
sufs.insert_many(sfs)

In [448]:
for w in ws:
    ss = []
    for s in w['sufs']:
        ss.append(sufs.find({'suf': s[0], 'gloss': s[1]})[0]['_id'])
    w['sufs'] = ss
    w['lemma'] = bases.find({'base': w['lemma'][0], 'trans': w['lemma'][1], 'pos': w['lemma'][2]})[0]['_id']

In [449]:
words.insert_many(ws)

In [450]:
for w in tks:
    #print(w)
    w['word'] = words.find(
        {
            'kor': w['word'][0],
            'pos': w['word'][1],
            'lemma': bases.find({'base': w['word'][2][0], 'trans': w['word'][2][1], 'pos': w['word'][2][2]})[0]['_id'],
            'sufs': [sufs.find({'suf': s[0], 'gloss': s[1]})[0]['_id'] for s in w['word'][3]]
        }
    )[0]['_id']
    w['page'] = pages.find({'page_num': int(w['page'][:-1]), 'page_let':w['page'][-1]})[0]['_id']

In [451]:
tokens.insert_many(tks)

### Демонстрация коллекций

In [452]:
from pprint import pprint

for i in pages.find()[:2]:
    pprint(i)

{'_id': ObjectId('624431d50051766d9df58933'),
 'lines': ['夫부와 婦부\ue285 두 姓셩의 合합홈이라',
           '生\ue9a2民민의 비로 \ue98b이며 萬만福복의',
           '근원이니 듕\ue57b를 行\uf55b\uf537야 혼인을 의논\uf537며',
           '폐\ue669을 드리고 친히 마즘은 그',
           '別별홈을 두터이홈이라 이런 故고로',
           '妻쳐를 娶\uf325호\ue3a8 同동姓셩을 娶\uf325티',
           '아니\uf537며 宮궁室실을 지으\ue3a8 안히며'],
 'page_let': 'a',
 'page_num': 6}
{'_id': ObjectId('624431d50051766d9df58934'),
 'lines': ['밧\uea35 분변\uf537야 男남子\uf1fc\ue285 밧긔 이셔',
           '안흘 니르디 아니\uf537고 婦부人인은 안\uf550',
           '이셔 밧\uea35 니르디 아니\uf537\ue283니 진실로',
           '能능히 莊장으로\ue73b 涖니\uf537야\ue73b 하\ue288의',
           '健건\uf53a 道도를 體톄\uf537고 부드러옴으로\ue73b',
           '正졍히 \uf537야 \ue73b \uea71희 順슌\uf53a',
           '義의를 니으면 집 道도ㅣ 正졍\uf537려니와'],
 'page_let': 'b',
 'page_num': 6}


In [453]:
for i in tokens.find()[:2]:
    pprint(i)

{'_id': ObjectId('624431d90051766d9df58c5b'),
 'line': 1,
 'line_pos': 1,
 'page': ObjectId('624431d50051766d9df58933'),
 'word': ObjectId('624431d70051766d9df58c1e')}
{'_id': ObjectId('624431d90051766d9df58c5c'),
 'line': 1,
 'line_pos': 2,
 'page': ObjectId('624431d50051766d9df58933'),
 'word': ObjectId('624431d70051766d9df58b66')}


In [454]:
for i in words.find()[:2]:
    pprint(i)

{'_id': ObjectId('624431d70051766d9df58ad1'),
 'kor': '니으면',
 'lemma': ObjectId('624431d50051766d9df58964'),
 'pos': 'P',
 'sufs': [ObjectId('624431d50051766d9df58a81')]}
{'_id': ObjectId('624431d70051766d9df58ad2'),
 'kor': '우희',
 'lemma': ObjectId('624431d50051766d9df58a66'),
 'pos': 'N',
 'sufs': [ObjectId('624431d50051766d9df58ab7')]}


In [455]:
for i in bases.find()[:2]:
    pprint(i)

{'_id': ObjectId('624431d50051766d9df58942'),
 'base': 'hoy-tey',
 'pos': 'N',
 'trans': 'baby'}
{'_id': ObjectId('624431d50051766d9df58943'),
 'base': 'cyel-cyel-ho',
 'pos': 'P',
 'trans': 'eager, ardent'}


In [456]:
for i in sufs.find()[:2]:
    pprint(i)

{'_id': ObjectId('624431d50051766d9df58a77'), 'gloss': 'PROB', 'suf': 'li'}
{'_id': ObjectId('624431d50051766d9df58a78'), 'gloss': '?', 'suf': 'thyey-lwo'}


### Update

In [458]:
for i in sufs.find({'gloss': '?'}):
    pprint(i)

{'_id': ObjectId('624431d50051766d9df58a78'), 'gloss': '?', 'suf': 'thyey-lwo'}
{'_id': ObjectId('624431d50051766d9df58a85'), 'gloss': '?', 'suf': 'l-ti-ni-la'}
{'_id': ObjectId('624431d50051766d9df58a8a'), 'gloss': '?', 'suf': 'nun'}
{'_id': ObjectId('624431d50051766d9df58a9a'), 'gloss': '?', 'suf': 'e'}
{'_id': ObjectId('624431d50051766d9df58aa0'), 'gloss': '?', 'suf': 'twoy'}
{'_id': ObjectId('624431d50051766d9df58aaf'), 'gloss': '?', 'suf': 'le-pse'}


In [459]:
sufs.update_many(
  {'gloss': '?'},
  {'$set': {'gloss': 'UNK'}}
)

In [460]:
for i in sufs.find({'gloss': 'UNK'}, projection={'_id': False}):
    pprint(i)

{'gloss': 'UNK', 'suf': 'thyey-lwo'}
{'gloss': 'UNK', 'suf': 'l-ti-ni-la'}
{'gloss': 'UNK', 'suf': 'nun'}
{'gloss': 'UNK', 'suf': 'e'}
{'gloss': 'UNK', 'suf': 'twoy'}
{'gloss': 'UNK', 'suf': 'le-pse'}


### Aggregate и все остальное

#### Частотность суффиксов. Создание нового ключа для ее хранения в коллекции суффиксов

In [461]:
for i in words.aggregate([
    
    {
        '$unwind': '$sufs'
    },
    {
        '$lookup': {
            'from': 'sufixes',
            'localField': 'sufs',
            'foreignField':'_id',
            'as': 'sufxs'
        }},
    {'$unwind': '$sufxs'},
    {
        '$group': {
            '_id': '$sufs',
            'suf': {'$first':'$sufxs.suf'},
            'gloss': {'$first':'$sufxs.gloss'},
            'count': { '$sum': 1 }
        }
    },
    {
        '$sort': {'count': -1}
    },
    {'$project': {'_id': 0, 'suf': 1, 'gloss': 1, 'count': 1}},

    ]):
    sufs.find_one_and_update({'suf': i['suf'], 'gloss': i['gloss']},
                             {'$set': {'freq': i['count']}})

In [462]:
for i in sufs.find()[:5]:
    print('Suffix: ' + i['suf'] + ', gloss: ' + i['gloss'] + ', freq: ' + str(i['freq']))

Suffix: li, gloss: PROB, freq: 7
Suffix: thyey-lwo, gloss: UNK, freq: 2
Suffix: oy, gloss: LOC, freq: 2
Suffix: un, gloss: TOP, freq: 6
Suffix: l-ti-ni, gloss: SHOULD, freq: 5


#### Insert и Delete

In [419]:
zero_suf = [
    {'suf': 'CAUS', 'gloss': 'lsoy', 'freq': 0},
    {'suf': 'DECL', 'gloss': 'ta', 'freq': 0}
]

sufs.insert_many(zero_suf)
for i in sufs.find({'freq': 0}):
    pprint(i)

{'_id': ObjectId('62442eed0051766d9df5842d'),
 'freq': 0,
 'gloss': 'lsoy',
 'suf': 'CAUS'}
{'_id': ObjectId('62442eed0051766d9df5842e'),
 'freq': 0,
 'gloss': 'ta',
 'suf': 'DECL'}


In [420]:
sufs.delete_many({'freq': {'$eq': 0}}) 
for i in sufs.find({'freq': 0}):
    pprint(i)

Группировка суффиксов по глоссам

In [422]:
for i in sufs.aggregate([
    
    {
        '$group': {
            '_id': '$gloss',
            'sufs': {'$push': '$suf'},
            'count': { '$sum': 1 }
        }
    },
    {'$match': {'count': {'$gt': 1}}},
    {'$sort': {'count': -1}},
    ]):
    print('Gloss: ' + i['_id'] + ', suffixes: ' + ', '.join(i['sufs']) + ', number of suffixes: ' + str(i['count']))

Gloss: CONC, suffixes: ni-wa, u-toy, ke-tun, wotoy, toy, n-ti-la, wo-toy, n-tuy, non-ti-la, number of suffixes: 9
Gloss: CV, suffixes: e, ya, ya-pse, ya-sa, (a), a, number of suffixes: 6
Gloss: LOC, suffixes: oy, ul, ey, yey, ay, uy, number of suffixes: 6
Gloss: UNK, suffixes: thyey-lwo, l-ti-ni-la, nun, e, twoy, le-pse, number of suffixes: 6
Gloss: HONSub, suffixes: si-wo, wo-sya, si, wo-si, number of suffixes: 4
Gloss: INST, suffixes: u-lwo-pse, lwo, u-lwo, lwo-pse, number of suffixes: 4
Gloss: COND, suffixes: u-myen, khe-tun, myen, ke-tun, number of suffixes: 4
Gloss: PIND, suffixes: un, n, nun, non, number of suffixes: 4
Gloss: NEG, suffixes: thi, u-ti, ti, number of suffixes: 3
Gloss: SHOULD, suffixes: l-ti-ni, l-ti-la, ul-ti-ni, number of suffixes: 3
Gloss: COOR, suffixes: mye, kwo, u-mye, number of suffixes: 3
Gloss: QUOT, suffixes: toy, wo-toy, number of suffixes: 2
Gloss: ACC, suffixes: lul, ul, number of suffixes: 2
Gloss: DAT, suffixes: ey, uy-key, number of suffixes: 2
Glos

#### Глоссировка

In [351]:
for i in tokens.aggregate([
    {
        '$lookup': {
            'from': 'words',
            'localField': 'word',
            'foreignField':'_id',
            'as': 'word'
        }},
    {"$unwind": "$word"},
    {
        '$lookup': {
            'from': 'bases',
            'localField': 'word.lemma',
            'foreignField':'_id',
            'as': 'base'
        }},
    {"$unwind": "$base"},
    {'$unwind': '$word.sufs'},
    {
        '$lookup': {
            'from': 'sufixes',
            'localField': 'word.sufs',
            'foreignField':'_id',
            'as': 'sufxs'
        }},
    {"$unwind": "$sufxs"},
    { "$group": {
        "_id": {
            "page": "$page",
            "line": "$line",
            "line_pos": "$line_pos"},
        "gloss": {"$push": "$sufxs.gloss"},
        "trans": {"$first": "$base.trans"},
        }},
    {
        '$lookup': {
            'from': 'pages',
            'localField': '_id.page',
            'foreignField':'_id',
            'as': 'page'
        }},
    {"$unwind": '$page'},
    {"$sort": {'page.page_num': 1, 'page.page_let': 1, "_id.line": 1, "_id.line_pos": 1}},
    { "$group": {
        "_id": {
            "page_num": "$page.page_num",
            "page_let": "$page.page_let",
            "line": "$_id.line"},
        "gloss": {"$push": "$gloss"},
        "trans": {"$push": "$trans"},
        "lines": {"$first": '$page.lines'}}},
    {"$sort": {'_id.page_num': 1, '_id.page_let': 1, "_id.line": 1}},
    {"$limit": 3}
        ,]):
    print('***')
    print(str(i['_id']['page_num']) + i['_id']['page_let'] + ' ' + str(i['_id']['line']))
    print('***')
    print(i['lines'][i['_id']['line'] - 1])
    line = []
    for trans, gl in zip(i['trans'], i['gloss']):
        if gl != ['-']:
            line.append(trans.split(',')[0] + '-'+ '-'.join(gl))
        else:
            line.append(trans.split(',')[0])
    print(' '.join(line))

***
6a 1
***
夫부와 婦부 두 姓셩의 合합홈이라
husband-COMIT wife-TOP two family-GEN unite-VOL-NMNZ-COP-DECL
***
6a 2
***
生民민의 비로 이며 萬만福복의
people-GEN birth-INST beginning-COOR great fortune-GEN
***
6a 3
***
근원이니 듕를 行야 혼인을 의논며
source-PARA matchmaking-ACC do-CV marriage-ACC discuss-COOR


#### Работа с текстами

In [424]:
from pymongo import TEXT

bases.create_index([("base", TEXT), ("trans", TEXT)])
sufs.create_index([("suf", TEXT)])

'suf_text'

In [425]:
for i in bases.aggregate(
   [
     {"$match": {"$text": {"$search": "\"husband and\""}}},
     {"$sort": {"score":{"$meta": "textScore"}}},
     {"$project": {"_id": 0, "base": 1, "trans": 1, "pos": 1}}
   ]
):
    pprint(i)

{'base': 'pwu-pwu', 'pos': 'N', 'trans': 'husband and wife'}


In [426]:
for i in bases.aggregate(
   [
     {"$match": {"$text": {"$search": "wife -husband"}}},
     {"$sort": {"score":{"$meta": "textScore"}}},
     {"$project": {"_id": 0, "base": 1, "trans": 1, "pos": 1}}
   ]
):
    pprint(i)

{'base': 'pwu', 'pos': 'N', 'trans': 'wife'}
{'base': 'chye', 'pos': 'N', 'trans': 'wife'}
{'base': 'an-hay', 'pos': 'N', 'trans': 'wife'}
{'base': 'pwu', 'pos': 'N', 'trans': "wife'"}


In [427]:
for i in bases.aggregate(
   [
     {"$match": {"$text": {"$search": "duty son"}}},
     {"$sort": {"score":{"$meta": "textScore"}}},
     {"$project": {"_id": 0, "base": 1, "trans": 1, "pos": 1}}
   ]
):
    pprint(i)

{'base': 'hywo-co', 'pos': 'N', 'trans': 'dutiful son'}
{'base': 'thyen-co', 'pos': 'N', 'trans': 'the son of God'}
{'base': 'two-li', 'pos': 'N', 'trans': 'duty, way'}


#### Regex

##### Строчки, содержащие 벋

In [428]:
for i in pages.find({'lines': {'$elemMatch': {'$regex': '.+벋.+'}}}, {'_id': 0, 'lines.$': 1}):
    print('\n'.join(i['lines']))

세 가짇 벋이오 해로오니 세 가짇
니를 벋며 便편佞령니를 벋면
샤 벋의게 믿브디 몯면


##### Перевод и корейское написание одной из форм основ на -ho

In [430]:
for i in bases.aggregate([
        {'$match': {'base': {'$regex': '.+\-ho'}}},
        {'$lookup': {
            'from': 'words',
            'localField': '_id',
            'foreignField':'lemma',
            'as': 'word'
        }},
        {'$unwind': '$word'},
        {'$project': {'_id': 0, 'word.kor': 1, 'trans': 1, 'base': 1}},
        {'$limit': 5}
    ]

):
    pprint(i)

{'base': 'cyel-cyel-ho',
 'trans': 'eager, ardent',
 'word': {'kor': '切졀切졀\uf537며'}}
{'base': 'thyey-ho',
 'trans': 'personally do, take shape',
 'word': {'kor': '體톄\uf537고'}}
{'base': 'kwong-kyeng-ho', 'trans': 'respectful', 'word': {'kor': '공경\uf537니'}}
{'base': 'toy-cyep-ho', 'trans': 'treat', 'word': {'kor': '\ue3a8졉홈을'}}
{'base': 'syey-sywu-ho',
 'trans': "wash one's face",
 'word': {'kor': '셰슈\uf537며'}}
